In [67]:
import urllib
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from os.path import join, exists
from os import  remove, makedirs
from time import gmtime, strftime

pd_symbol = 'LH'
symbol_list = [pd_symbol, 'ASP', 'CPALL', 'BAFS', 'MINT', 'AOT', 'BEM', 'TU']

def getSymbol():
    url_string = "https://www.settrade.com/C13_MarketSummary.jsp?detail=SET100"
    
    page = urllib.request.urlopen(url_string).read()
    soup = BeautifulSoup(page, 'lxml')    
    table_element =soup.find('table', class_='table table-info table-hover')
    
    row_list =[]
    tr_list = table_element.findAll('tr')    
        
    for tr in tr_list:
                
        td_list = tr.findAll('td') 
        for td in td_list:
            row_list = np.append(row_list, td.find(text=True))
            break;
    print('table', row_list)
    return row_list
    
symbol_list = getSymbol()

# Example url
# https://www.set.or.th/set/historicaltrading.do?symbol=BBL&page=2&language=en&country=US&type=trading
def getTableData(symbol, page=1):
    if page > 3:
        page = 3 # limit at 3
      
    url_string = "https://www.set.or.th/set/historicaltrading.do?symbol={0}".format(symbol)
    url_string += '&page={0}&language=en&country=US&type=trading'.format(page-1)
        
    page = urllib.request.urlopen(url_string).read()           
    soup = BeautifulSoup(page, 'lxml')    
    table_element =soup.find('table', class_='table table-hover table-info')
    return table_element, url_string

def createDataFrame(table_element):    
    row_list =[]
    head_list = []
    
    if table_element is None:
        return None
    
    tr_list = table_element.findAll('tr')    
        
    for tr in tr_list:
        th_list = tr.findAll('th')
        if th_list is not None:
            for th in th_list:
                head_list.append(th.find(text=True))
                
        td_list = tr.findAll('td') 
         
        for td in td_list:
            row_list = np.append(row_list, td.find(text=True))        
        
    num_col = len(head_list)        
    total_col = int(len(row_list)/num_col)    
    row_list = np.reshape(row_list, (total_col, num_col) )       
    df=pd.DataFrame(columns = head_list, data = row_list)
    return df

def create_all_data(symbol, total_page=1):
    # get stock data from set.or.th web (total page)
    df = None
    for p in range(1, total_page+1):
        table_element, url_string = getTableData(symbol, page=p) 
        #print(url_string)
        df_temp = createDataFrame(table_element)
        if df is None:
            df = df_temp
        else:        
            df = df.append(df_temp)
    return df

DIR_SEC_CSV = "sec_set_price"
def writeCSVFile(df, symbol, output_path=DIR_SEC_CSV, include_index = False):
    csv_file = "{}.csv".format(join(output_path, symbol))    
    df.to_csv(csv_file, index = include_index)    
        
def removeOldFile(symbol, output_path=DIR_SEC_CSV):
    csv_file = "{}.csv".format(join(output_path, symbol))    
    if exists(output_path) == False:
        makedirs(output_path)
    if exists(csv_file):
        remove(csv_file)


# symbol_list = ['AOT', 'ASP', 'BAFS', 'CPALL', 'PM']
for symbol in symbol_list:
    df = create_all_data(symbol, total_page = 3)
#     print('\n********* %s **********' % symbol)        
#     print(df.head(20))
        
    # save csv files (all stock data)
    removeOldFile(symbol) # clear old files
    writeCSVFile(df, symbol)

print('test')

table ['AAV' 'ACE' 'ADVANC' 'AEONTS' 'AMATA' 'AOT' 'AP' 'AWC' 'BANPU' 'BBL'
 'BCH' 'BCP' 'BCPG' 'BDMS' 'BEC' 'BEM' 'BGRIM' 'BH' 'BJC' 'BPP' 'BTS'
 'CBG' 'CENTEL' 'CHG' 'CK' 'CKP' 'COM7' 'CPALL' 'CPF' 'CPN' 'CRC' 'DOHOME'
 'DTAC' 'EA' 'EGCO' 'EPG' 'ESSO' 'GFPT' 'GLOBAL' 'GPSC' 'GULF' 'GUNKUL'
 'HANA' 'HMPRO' 'INTUCH' 'IRPC' 'IVL' 'JAS' 'JMT' 'KBANK' 'KCE' 'KKP'
 'KTB' 'KTC' 'LH' 'MAJOR' 'MEGA' 'MINT' 'MTC' 'ORI' 'OSP' 'PLANB' 'PRM'
 'PSH' 'PTG' 'PTT' 'PTTEP' 'PTTGC' 'QH' 'RATCH' 'RBF' 'RS' 'SAWAD' 'SCB'
 'SCC' 'SCGP' 'SGP' 'SIRI' 'SPALI' 'SPRC' 'STA' 'STEC' 'SUPER' 'TASCO'
 'TCAP' 'THANI' 'TISCO' 'TKN' 'TMB' 'TOA' 'TOP' 'TPIPP' 'TQM' 'TRUE' 'TTW'
 'TU' 'TVO' 'VGI' 'WHA' 'WHAUP']
test


In [68]:

import os


DIR_CURRENT = os.path.dirname(os.path.abspath("__file__"))

def ema(df, periods=12):
    return df.ewm(span=periods, adjust=False, min_periods=0, ignore_na=False).mean()
#     return df.ewm(span=periods, adjust=True, min_periods=0, ignore_na=False).mean()


def loadPriceData(symbol_list, dates,  base_dir=DIR_SEC_CSV):
    return loadManySymbols(symbol_list, dates, 'Close', base_dir)

def loadManySymbols(symbols, dates, column_name, base_dir):
    df = pd.DataFrame(index=dates)	# empty data frame that has indexs as dates
        
    base_dir = join(DIR_CURRENT,base_dir)
    for symbol in symbols:
        csv_file = os.path.join(base_dir, symbol + '.csv')
        df_temp = pd.read_csv(csv_file, index_col='Date',
            parse_dates=False, usecols=['Date', column_name], 
            na_values=['nan'])
        df_temp = df_temp.rename(columns={column_name: symbol})
        df = df.join(df_temp) # left join by default
        df = df.dropna()
#         print('df test', df)
    return df

def average_convergence(df, period_low=26, period_fast=12):
    emaslow = ema(df, period_low)
    emafast = ema(df, period_fast)
    return (emaslow, emafast, emafast - emaslow)

def signal_MACD(df_MACD, periods=9):
	return ema(df_MACD, periods)

startDate = '2017-03-01'
endDate = strftime("%Y-%m-%d", gmtime())

# endDate = '2020-07-31'
# dates = pd.date_range(startDate, endDate)[::-1]
dates = pd.date_range(startDate, endDate).strftime('%d/%m/%Y')

print('dates', dates)
symbols = ['AOT','BBL', 'ASP','BAFS', 'CPALL', 'PM']
df = loadPriceData(symbols, dates)

print("+++++EMA 15+++++")
ema15 = ema(df, 15)
print(ema15.tail())
# ema45 = ema(df, 45)
# ema100 = ema(df, 100)

print("+++++MACD+++++")
ema26, ema12, macd = average_convergence(df)
print(macd.tail())

print("+++++EMA 12+++++")
print(ema12.tail())

print("+++++EMA 26+++++")
print(ema26.tail())

# print("+++++Signal MACD+++++")
# signal = signal_MACD(macd)
# print(signal.head(10))


dates Index(['01/03/2017', '02/03/2017', '03/03/2017', '04/03/2017', '05/03/2017', '06/03/2017', '07/03/2017', '08/03/2017', '09/03/2017', '10/03/2017',
       ...
       '16/11/2020', '17/11/2020', '18/11/2020', '19/11/2020', '20/11/2020', '21/11/2020', '22/11/2020', '23/11/2020', '24/11/2020', '25/11/2020'], dtype='object', length=1366)
+++++EMA 15+++++
                  AOT         BBL       ASP       BAFS      CPALL        PM
17/08/2020  53.098318  103.788750  1.774611  21.400159  67.409432  6.867369
18/08/2020  53.367278  104.190157  1.780285  21.462639  67.077003  6.871448
19/08/2020  53.352618  104.353887  1.783999  21.454809  66.692378  6.875017
20/08/2020  53.277291  104.122151  1.780999  21.397958  66.230831  6.890640
21/08/2020  53.367629  104.106882  1.780874  21.373213  65.795727  6.898060
+++++MACD+++++
                 AOT       BBL       ASP      BAFS     CPALL        PM
17/08/2020 -1.259756 -1.049503  0.027332 -0.441791 -0.204677  0.081345
18/08/2020 -0.981487 -0.71270

In [69]:
from stock_pandas import StockDataFrame
df_symbol = loadPriceData([pd_symbol], dates)
print('df_symbol', df_symbol.tail(20))
stock = StockDataFrame(df_symbol)

stock.alias('close', pd_symbol)
stock[[
    'ma:5',
    'ma:10',
    'ma:15',
    'ma:30'
]]



df_symbol               LH
28/10/2020  6.50
29/10/2020  6.40
30/10/2020  6.40
02/11/2020  6.45
03/11/2020  6.55
04/11/2020  6.55
05/11/2020  6.75
06/11/2020  6.70
09/11/2020  6.95
10/11/2020  7.20
11/11/2020  7.40
12/11/2020  7.30
13/11/2020  7.40
16/11/2020  7.50
17/11/2020  7.60
18/11/2020  7.75
19/11/2020  7.70
20/11/2020  7.80
23/11/2020  7.95
24/11/2020  7.85


,"ma:5,close","ma:10,close","ma:15,close","ma:30,close"
01/06/2020,NaN,NaN,NaN,NaN
02/06/2020,NaN,NaN,NaN,NaN
04/06/2020,NaN,NaN,NaN,NaN
05/06/2020,NaN,NaN,NaN,NaN
08/06/2020,7.86,NaN,NaN,NaN
09/06/2020,7.97,NaN,NaN,NaN
10/06/2020,8.04,NaN,NaN,NaN
11/06/2020,8.02,NaN,NaN,NaN
12/06/2020,7.95,NaN,NaN,NaN
15/06/2020,7.80,7.830,NaN,NaN


In [70]:

stock['macd'].tail(20)

stock['macd.dif']

# # macd signal band, which is a shortcut for stock['macd.signal']
# stock['macd.s']
# stock['macd.signal']
# stock['macd.dea']

# # macd histogram band, which is equivalent to stock['macd.h']
# stock['macd.histogram']
# stock['macd.h']
# stock['macd.macd']

01/06/2020         NaN
02/06/2020         NaN
04/06/2020         NaN
05/06/2020         NaN
08/06/2020         NaN
09/06/2020         NaN
10/06/2020         NaN
11/06/2020         NaN
12/06/2020         NaN
15/06/2020         NaN
16/06/2020         NaN
17/06/2020         NaN
18/06/2020         NaN
19/06/2020         NaN
22/06/2020         NaN
23/06/2020         NaN
24/06/2020         NaN
25/06/2020         NaN
26/06/2020         NaN
29/06/2020         NaN
30/06/2020         NaN
01/07/2020         NaN
02/07/2020         NaN
03/07/2020         NaN
07/07/2020         NaN
08/07/2020    0.018196
09/07/2020    0.014281
10/07/2020    0.007486
13/07/2020   -0.008716
14/07/2020   -0.028526
15/07/2020   -0.032697
16/07/2020   -0.039258
17/07/2020   -0.040231
20/07/2020   -0.040527
21/07/2020   -0.025322
22/07/2020   -0.016897
23/07/2020   -0.006335
24/07/2020   -0.016957
29/07/2020   -0.028881
30/07/2020   -0.049342
31/07/2020   -0.060960
03/08/2020   -0.073200
04/08/2020   -0.074237
05/08/2020 

In [71]:
stock[['macd', 'macd.s', 'macd.h', 'macd.dif']].tail(50)

,"macd:12,26","macd.signal:12,26,9","macd.histogram:12,26,9","macd:12,26"
14/09/2020,-0.096180,-0.070949,-0.050463,-0.096180
15/09/2020,-0.098296,-0.076419,-0.043756,-0.098296
16/09/2020,-0.098833,-0.080902,-0.035863,-0.098833
17/09/2020,-0.106075,-0.085936,-0.040277,-0.106075
18/09/2020,-0.114514,-0.091652,-0.045724,-0.114514
21/09/2020,-0.127773,-0.098876,-0.057792,-0.127773
22/09/2020,-0.144657,-0.108033,-0.073249,-0.144657
23/09/2020,-0.148280,-0.116082,-0.064396,-0.148280
24/09/2020,-0.157386,-0.124343,-0.066085,-0.157386
25/09/2020,-0.162725,-0.132019,-0.061411,-0.162725


In [72]:
stock['increase:(ma:20,close),3'].tail(20)

28/10/2020    False
29/10/2020    False
30/10/2020    False
02/11/2020    False
03/11/2020    False
04/11/2020    False
05/11/2020    False
06/11/2020    False
09/11/2020    False
10/11/2020    False
11/11/2020    False
12/11/2020     True
13/11/2020     True
16/11/2020     True
17/11/2020     True
18/11/2020     True
19/11/2020     True
20/11/2020     True
23/11/2020     True
24/11/2020     True
Name: increase:(ma:20,close),3,1, dtype: bool

In [73]:

df_signal = None
for symbol in symbol_list:
    df_symbol = loadPriceData([symbol], dates)
    stock_signal = StockDataFrame(df_symbol)
    stock_signal.alias('close', symbol)
#     print('stock_signal', stock_signal['boll'].tail(10))
#     df_temp = stock_signal[['macd.signal \\ macd']].rename(columns={'macd.signal:12,26,9\macd:12,26': symbol})
    df_temp = stock_signal[['macd \\ macd.signal']].rename(columns={'macd:12,26\macd.signal:12,26,9': symbol})
    if df_signal is None:
        df_signal = df_temp
    else:
        df_signal = df_signal.join(df_temp)
    
df_signal.tail(10)
    

/Users/nopadonmanlin/opt/anaconda3/lib/python3.7/site-packages/stock_pandas/common.py:184: RuntimeWarning: invalid value encountered in less
  less = right < left


,AAV,ACE,ADVANC,AEONTS,AMATA,AOT,AP,AWC,BANPU,BBL,BCH,BCP,BCPG,BDMS,BEC,BEM,BGRIM,BH,BJC,BPP,BTS,CBG,CENTEL,CHG,CK,CKP,COM7,CPALL,CPF,CPN,CRC,DOHOME,DTAC,EA,EGCO,EPG,ESSO,GFPT,GLOBAL,GPSC,GULF,GUNKUL,HANA,HMPRO,INTUCH,IRPC,IVL,JAS,JMT,KBANK,KCE,KKP,KTB,KTC,LH,MAJOR,MEGA,MINT,MTC,ORI,OSP,PLANB,PRM,PSH,PTG,PTT,PTTEP,PTTGC,QH,RATCH,RBF,RS,SAWAD,SCB,SCC,SCGP,SGP,SIRI,SPALI,SPRC,STA,STEC,SUPER,TASCO,TCAP,THANI,TISCO,TKN,TMB,TOA,TOP,TPIPP,TQM,TRUE,TTW,TU,TVO,VGI,WHA,WHAUP
11/11/2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
12/11/2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
13/11/2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
16/11/2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17/11/2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
18/11/2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [74]:
def loadPriceData2(symbol_list, dates,  base_dir=DIR_SEC_CSV):
    return loadManySymbols2(symbol_list, dates, 'Close', base_dir)

def loadManySymbols2(symbols, dates, column_name, base_dir):
    df = pd.DataFrame(index=dates)	# empty data frame that has indexs as dates
        
    base_dir = join(DIR_CURRENT,base_dir)
    for symbol in symbols:
        csv_file = os.path.join(base_dir, symbol + '.csv')
        df_temp = pd.read_csv(csv_file, index_col='Date',
            parse_dates=False,
            na_values=['nan'])
        df_temp = df_temp.rename(columns={column_name: symbol})
        df = df.join(df_temp) # left join by default
        df = df.dropna()
#         print('df test', df)
    return df


df_signal = None
for symbol in symbol_list:
    df_symbol = loadPriceData2([symbol], dates)
    stock_signal = StockDataFrame(df_symbol)
    stock_signal.alias('close', symbol)
    stock_signal.alias('low', 'Low')
    stock_signal.alias('high', 'High')
#     print('stock_signal', stock_signal['boll'].tail(10))
#     df_temp = stock_signal[['macd.signal \\ macd']].rename(columns={'macd.signal:12,26,9\macd:12,26': symbol})
    df_temp = stock_signal[['kdj.d / kdj.k']].rename(columns={'kdj.d:9,3,3,50.0/kdj.k:9,3,50.0': symbol})
    if df_signal is None:
        df_signal = df_temp
    else:
        df_signal = df_signal.join(df_temp)
    
df_signal.tail(10)

,AAV,ACE,ADVANC,AEONTS,AMATA,AOT,AP,AWC,BANPU,BBL,BCH,BCP,BCPG,BDMS,BEC,BEM,BGRIM,BH,BJC,BPP,BTS,CBG,CENTEL,CHG,CK,CKP,COM7,CPALL,CPF,CPN,CRC,DOHOME,DTAC,EA,EGCO,EPG,ESSO,GFPT,GLOBAL,GPSC,GULF,GUNKUL,HANA,HMPRO,INTUCH,IRPC,IVL,JAS,JMT,KBANK,KCE,KKP,KTB,KTC,LH,MAJOR,MEGA,MINT,MTC,ORI,OSP,PLANB,PRM,PSH,PTG,PTT,PTTEP,PTTGC,QH,RATCH,RBF,RS,SAWAD,SCB,SCC,SCGP,SGP,SIRI,SPALI,SPRC,STA,STEC,SUPER,TASCO,TCAP,THANI,TISCO,TKN,TMB,TOA,TOP,TPIPP,TQM,TRUE,TTW,TU,TVO,VGI,WHA,WHAUP
11/11/2020,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False
12/11/2020,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
13/11/2020,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
16/11/2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17/11/2020,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,True,True,True,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,True,False,False,True,False,False,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,False,True,False,False,True,False,True,False,False,False,True,False,False
18/11/2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,Fal

In [75]:
inspec_symbol = 'chg'
df_symbol = loadPriceData2([inspec_symbol], dates)
# print('df_symbol', df_symbol)
stock = StockDataFrame(df_symbol)

stock.alias('close', inspec_symbol)
stock.alias('low', 'Low')
stock.alias('high', 'High')
# stock.alias('low', 'Low')
stock[['macd \\ macd.signal', 'macd.signal \\ macd','kdj.d / kdj.k', 'close', 'ma:15','change:(ma:5)','ma:15',
       'increase:(ma:5,close),3','boll', 'boll.u', 'boll.l','change:(ma:5)']].rename(columns={'macd:12,26\macd.signal:12,26,9':'macd',
                                                                                              'macd.signal:12,26,9\macd:12,26':'signal',
                                                                                              'kdj.k:9,3,50.0/kdj.d:9,3,3,50.0':'k/d',
                                                                                             }).tail(50)

/Users/nopadonmanlin/opt/anaconda3/lib/python3.7/site-packages/stock_pandas/common.py:184: RuntimeWarning: invalid value encountered in less
  less = right < left


,macd,signal,"kdj.d:9,3,3,50.0/kdj.k:9,3,50.0",chg,"ma:15,close","change:(ma:5,close),2","ma:15,close","increase:(ma:5,close),3,1","boll:20,close","boll.upper:20,2.0,close","boll.lower:20,2.0,close","change:(ma:5,close),2"
14/09/2020,False,False,False,2.48,2.581333,-7.836991e-03,2.581333,False,2.582,2.698550,2.465450,-7.836991e-03
15/09/2020,False,False,False,2.48,2.569333,-9.478673e-03,2.569333,False,2.582,2.698550,2.465450,-9.478673e-03
16/09/2020,False,False,False,2.58,2.561333,3.189793e-03,2.561333,False,2.583,2.699121,2.466879,3.189793e-03
17/09/2020,False,False,False,2.48,2.550667,-6.359300e-03,2.550667,False,2.575,2.696244,2.453756,-6.359300e-03
18/09/2020,False,False,True,2.48,2.541333,0.000000e+00,2.541333,False,2.567,2.691113,2.442887,0.000000e+00
21/09/2020,False,False,False,2.46,2.534667,-1.600000e-03,2.534667,False,2.560,2.691453,2.428547,-1.600000e-03
22/09/2020,False,False,False,2.44,2.526667,-3.205128e-03,2.526667,False,2.549,2.681951,2.416049,-3.205128e-03
23/09/2020,False,False,False,2.46,2.517333,-9.646302e-03,2.517333,False,2.537,2.655844,2.418156,-9.646302e-03
24/09/2020,False,False,False,2.44,2.509333,-3.246753e-03,2.509333,False,2.527,2.643121,2.410879,-3.246753e-03
25/09/2020,False,False,False,2.44,2.500000,-3.257329e-03,2.500000,False,2.518,2.631772,2.404228,-3.257329e-03
